In [ ]:
import numpy as np
import geopandas as gpd
from IPython.display import display
from shapely.geometry import Polygon
from utils import process_and_save_data
from model import EnhancedLandUsePredictionModel
from views import create_band_viewer, create_classification_viewer, create_spectral_indices_viewer, create_map_with_slider, analyze_time_series, plot_time_series, create_prediction_map_with_slider, create_prediction_viewer, visualize_predictions, ModelAnalysis


In [ ]:
roi_coords = [
    [75.58679011123377, 26.725580815770467],
    [76.00152399795252, 26.725580815770467],
    [76.00152399795252, 27.08929110630265],
    [75.58679011123377, 27.08929110630265],
    [75.58679011123377, 26.725580815770467]
]

roi_polygon = Polygon(roi_coords)
roi_gdf = gpd.GeoDataFrame(index=[0], crs='EPSG:4326', geometry=[roi_polygon])



In [ ]:
# Define file paths
data_dir = 'DataSet'
processed_dir = 'Processed_DataSet'
bounds = [[26.725580815770467, 75.58679011123377], 
         [27.08929110630265, 76.00152399795252]]

# Analyze time series
results = analyze_time_series(data_dir)

# Plot the time series results
plot_time_series(results)


In [ ]:
all_processed_data = process_and_save_data(data_dir, processed_dir)


In [ ]:
band_viewer = create_band_viewer('DataSet')
display(band_viewer)


In [ ]:
spectral_viewer = create_spectral_indices_viewer(all_processed_data)
display(spectral_viewer)


In [ ]:
classification_viewer = create_classification_viewer(all_processed_data)
display(classification_viewer)


In [ ]:
Map = create_map_with_slider(all_processed_data, bounds, roi_gdf)
Map


In [ ]:

model = EnhancedLandUsePredictionModel(
        spatial_smoothing=1.0,
        window_size=3,
        n_estimators=50
    )

model.fit(all_processed_data, verbose=True, batch_size=10)

model.save_model('land_use_model.joblib')



In [ ]:
model = EnhancedLandUsePredictionModel.load_model('land_use_model.joblib')


In [ ]:
predictions = model.predict_future(
        historical_window=all_processed_data[-24:],
        steps=3,
        verbose=True
    )

In [ ]:
model.plot_prediction_results(predictions)

In [ ]:
model.plot_prediction(predictions[0])

In [ ]:
visualize_predictions(predictions)

In [ ]:
for pred in predictions:
    print(f"\nPrediction for {pred['date_str']}:")
    unique, counts = np.unique(pred['classification'], return_counts=True)
    for cls, count in zip(unique, counts):
        print(f"Class {cls} ({model.class_mapping[cls]}): {count} pixels")
    print(f"Mean confidence: {np.mean(pred['confidence']):.3f}")

In [ ]:
analyzer = ModelAnalysis()

In [ ]:
print("\nAnalyzing class distribution...")
class_dist = analyzer.analyze_class_distribution(predictions)
print(class_dist)
        
print("\nAnalyzing prediction confidence...")
confidence_analysis = analyzer.analyze_confidence(predictions)
print(confidence_analysis)
        
print("\nAnalyzing spatial changes...")
spatial_changes = analyzer.analyze_spatial_changes(predictions)
print(spatial_changes)

In [ ]:
total_data = all_processed_data + predictions

In [ ]:
P_map = create_prediction_map_with_slider(total_data, bounds, roi_gdf)
P_map

In [ ]:
P_view = create_prediction_viewer(total_data)
display(P_view)